In [342]:
import pandas as pd

import os
from dotenv import load_dotenv, find_dotenv

In [343]:
id = 101

In [344]:
%%script false
# Ignore
# Config which houses the credentials for skillplus database
load_dotenv(find_dotenv())
login = os.getenv('login')
pw = os.getenv('password')
server = os.getenv('server')
db = os.getenv('database')

connection_string = r'Driver={ODBC Driver 18 for SQL Server};Server='+ server +';Database='+ db +';Uid=' + login + ';Pwd=' + pw + ';Encrypt=yes;'

Couldn't find program: 'false'


In [345]:
%%script false
# Ignore
id = 105

query = f'''
SELECT mk.timestamp_s, mk.activityid, hr.*, imu.*
FROM [dbo].[imudata] as imu
LEFT JOIN [dbo].[masterkey] as mk ON mk.compositekey = imu.compositekey
LEFT JOIN [dbo].[hrdata] as hr ON mk.compositekey = hr.compositekey
WHERE mk.id = '{id}';
'''

def extract(query):
    '''
    Runs the query against skillplus database
    '''
    conn = pyodbc.connect(connection_string)     
    dataset = pd.read_sql(query, conn)
    return dataset

raw_data = extract(query).sort_values(by = 'timestamp_s').reset_index().drop(columns='index')

Couldn't find program: 'false'


In [346]:
with open('resources\PAMAP2_Dataset\headers.csv', encoding = 'UTF-8') as file:
    headers = file.read().split(',')

raw_data = pd.read_csv(f'resources\PAMAP2_Dataset\Protocol\subject{id}.dat', delimiter=' ', names = headers)

subjectInfo = pd.read_excel('resources\PAMAP2_Dataset\SubjectInfo.xlsx')
subject = subjectInfo[subjectInfo['id']==id]

activity_map = {
    1: 'lying',
    2: 'sitting',
    3: 'standing',
    4: 'walking',
    5: 'running',
    6: 'cycling',
    7: 'Nordic walking',
    9: 'watching TV',
    10: 'computer work',
    11: 'car driving',
    12: 'ascending stairs',
    13: 'descending stairs',
    16: 'vacuuming',
    17: 'ironing',
    18: 'folding laundry',
    19: 'house cleaning',
    20: 'playing soccer',
    24: 'rope jumping'
}

In [347]:
def activity_grouper(df):
    '''
    Groups timestamps into blocks of activities
    A block is a time period where the subject does 1 activity continously
    '''
    activities = df['activityid']
    output = []
    counter = 1

    for i in range(len(activities)):
        if i == 0:
            output.append(counter)
        else:
            if activities[i-1] == activities[i]:
                output.append(counter)
            else:
                counter += 1
                output.append(counter)

    return output

raw_data['block'] = activity_grouper(raw_data)

In [348]:
%%script false
# Ignore
# imputing missing heart rates by finding the mid point/average between last and next heart rate
raw_hr = raw_data['heartRate_bpm']
output = []
previous_value = None
current_value = None
nan_counter = 0
for i in range(len(raw_hr)):
    if pd.isnull(raw_hr[i]):
        nan_counter += 1
    else:
        current_value = raw_hr[i]
        if previous_value == None:
            to_append = [current_value] * nan_counter
            to_append.append(current_value)
        else:
            average = (current_value + previous_value)/2
            to_append = [average] * nan_counter
            to_append.append(current_value)

        output.extend(to_append)
        nan_counter = 0  
        previous_value = current_value

raw_data.insert(loc = 3, column = 'imputedHeartRate_bpm', value = output)

Couldn't find program: 'false'


In [349]:
# converting acceleration given in ms^2 to g units for mad calculation
# https://www.convertunits.com/from/m/s%5E2/to/g-unit
# 1 ms^2 = 0.10197162129779283 g units
for i in raw_data.columns:
    if 'Accel' in i:
        conversion = 0.10197162129779283
        new_col_name = i.replace('ms2', 'gUnits')
        raw_data[new_col_name] = raw_data[i] * conversion

In [350]:
# https://journals.plos.org/plosone/article/file?id=10.1371/journal.pone.0134813&type=printable
# resultant acceleration (ri), which defines the magnitude of the acceleration vector 
raw_data['chestResultant_ms2'] = (raw_data['chestAccelAxis1_16g_ms2']**2 + raw_data['chestAccelAxis2_16g_ms2']**2 + raw_data['chestAccelAxis3_16g_ms2']**2)**0.5
raw_data['chestResultant_gUnits'] = (raw_data['chestAccelAxis1_16g_gUnits']**2 + raw_data['chestAccelAxis2_16g_gUnits']**2 + raw_data['chestAccelAxis3_16g_gUnits']**2)**0.5

In [351]:
def calc_vo2(df, block, epoch_s, age, restingHR):
    '''
    Calculates vo2 from acceleration and heart rate data
    '''
    vo2PerEpoch = []
    epoch_interval = epoch_s * 100
    epoch_counter = 1

    for i in range(0, (df['block']==block).sum(), epoch_interval):
        epoch_start = i
        blockactivity = df[df['block']==block]['activityid'].iloc[0]

        chest_cols = ['timestamp_s', 'activityid', 'chestAccelAxis1_16g_gUnits', 'chestAccelAxis2_16g_gUnits', 'chestAccelAxis3_16g_gUnits', 'chestResultant_gUnits']
        epochimu = df[df['block']==block][epoch_start:epoch_start + epoch_interval][chest_cols]
        epochhr = df[df['timestamp_s'].between(min(epochimu['timestamp_s']), max(epochimu['timestamp_s']))]['heartRate_bpm'].dropna()
        epochlength = len(epochimu)

        # https://journals.plos.org/plosone/article/file?id=10.1371/journal.pone.0134813&type=printable
        # calculation of mad for epoch
        epoch_mean_resultant = epochimu['chestResultant_gUnits'].mean()
        epochimu['chestResultantDeviation_gUnits'] = (epochimu['chestResultant_gUnits'] - epoch_mean_resultant).abs()
        epochmad = epochimu['chestResultantDeviation_gUnits'].mean()
        
        epochHrIndex = epochhr.mean() / restingHR

        # https://roderic.uv.es/bitstream/handle/10550/82752/152890.pdf?sequence=1
        # regression used to estimate vo2
        epochVo2 = 8.62121 + (29.10141 * epochmad) - (0.08096 * age) + (2.84826 * epochHrIndex) - (1.81686 * 0)
        epoch = f"epoch{epoch_counter}"

        vo2PerEpoch.append([epoch, blockactivity, epochlength, epochVo2])
        epoch_counter += 1
    
    return vo2PerEpoch

In [352]:
def vo2_conversion(df, weight):
    '''
    Converts vo2 into several formats (MET, kCal per hour and kCal per minute)
    VO2 -> MET https://www.omicsonline.org/articles-images/2157-7595-6-220-t003.html 1 MET = 3.5 ml kg1 min1 of O2
    MET -> kCal https://www.omicsonline.org/articles-images/2157-7595-6-220-t003.html 1 MET = 1 kcal kg-1 hr-1
    VO2 -> kCal https://www.ideafit.com/personal-training/how-to-calculate-calories-expended/#:~:text=Once%20VO2%20is%20in,by%20using%20this%20conversion%20factor.
    '''
    df['MET'] = df['vo2'] / 3.5
    df['kCalPerHour_MET'] = df['MET'] * weight
    df['kCalPerMin_MET'] = df['kCalPerHour_MET'] / 60
    df['kCalPerMin_vo2'] = ((df['vo2'] * weight)/1000) * 5
    df['totalkCal_MET'] = (df['epochLength']/100)*(df['kCalPerMin_MET']/60)
    df['totalkCal_vo2'] = (df['epochLength']/100)*(df['kCalPerMin_vo2']/60)

    return df

In [353]:
nonZero = raw_data[raw_data['activityid']!= 0]['block'].unique()

sums = pd.DataFrame(columns = ['block', 'activityid', 'duration_s', 'totalkCal_MET','kCalPerMin_MET', 'totalkCal_vo2', 'kCalPerMin_vo2'])

for i in nonZero:
    vo2PerEpoch = pd.DataFrame(calc_vo2(df=raw_data, block=i, epoch_s=5, age=subject['age'], restingHR=subject['restingHR']), columns = ['epoch', 'activityid', 'epochLength', 'vo2'])
    ee = vo2_conversion(df = vo2PerEpoch, weight = 73)
    
    totalEpoch_s = ee['epochLength'].sum()/100

    totalkCalBurn_MET = float(ee['totalkCal_MET'].sum())
    kCalPerMin_MET = totalkCalBurn_MET/(totalEpoch_s/60)
    totalkCalBurn_vo2 = float(ee['totalkCal_vo2'].sum())
    kCalPerMin_vo2 = totalkCalBurn_vo2/(totalEpoch_s/60)

    to_append = [i, ee['activityid'].iloc[0], totalEpoch_s, totalkCalBurn_MET, kCalPerMin_MET, totalkCalBurn_vo2, kCalPerMin_vo2]

    sums.loc[len(sums)] = to_append

sums

,block,activityid,duration_s,totalkCal_MET,kCalPerMin_MET,totalkCal_vo2,kCalPerMin_vo2
0,2.0,1.0,271.87,16.014259,3.534246,16.814972,3.710959
1,3.0,2.0,234.80,13.973995,3.570868,14.672695,3.749411
2,4.0,3.0,217.17,13.379608,3.696535,14.048589,3.881362
3,6.0,17.0,235.73,14.636305,3.725356,15.368120,3.911624
4,8.0,16.0,229.41,18.314050,4.789865,19.229753,5.029359
5,10.0,12.0,81.20,8.759870,6.472810,9.197863,6.796451
6,12.0,13.0,74.80,8.890134,7.131123,9.334640,7.487679
7,14.0,12.0,77.70,8.510548,6.571852,8.936075,6.900444
8,15.0,13.0,74.19,9.232119,7.466332,9.693725,7.839648
9,17.0,4.0,222.53,25.496647,6.874573,26.771479,7.218302


In [354]:
pivot = pd.pivot_table(sums, ['duration_s', 'totalkCal_MET'], 'activityid', aggfunc=sum)

In [355]:
pivot['avgkCal_MET']=pivot['totalkCal_MET']/(pivot['duration_s']/60)
pivot.index = pivot.index.map(activity_map)

In [356]:
pivot

,duration_s,totalkCal_MET,avgkCal_MET
activityid,,,
lying,271.87,16.014259,3.534246
sitting,234.80,13.973995,3.570868
standing,217.17,13.379608,3.696535
walking,222.53,25.496647,6.874573
running,212.65,43.829498,12.366658
cycling,235.75,18.820153,4.789859
Nordic walking,202.65,24.195910,7.163852
ascending stairs,158.90,17.270418,6.521240
descending stairs,148.99,18.122253,7.298041
